# 15. Frame Selected Matching

In [3]:
from src import *

/home/chunhei/miniconda3/envs/gradients_emo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Description

We look in this section for the capacity of matching individuals using frame selected gradients.

GOAL: Compute accuracy of frame selected gradients versus movie

## Table of Contents
- [Frame selected emotion gradients](#comparison)

In [4]:
mri_path400 = "./data/fmri414_compiled/"

<a class="anchor" id="comparison"></a>
## Frame selected (by emotion) gradients

### Defining Emotion Intensity

In [5]:
select = ['Anger','Guilt','WarmHeartedness', \
             'Disgust','Happiness','Fear','Regard','Anxiety', \
             'Satisfaction','Pride','Surprise','Love','Sad']

tmpdf = pd.read_csv('./data/emotion_compileBIDS.csv')

In [20]:
order_idx = {}
for movie_name400 in tqdm(os.listdir(mri_path400)):
    if "Rest" in movie_name400: continue        
    mri_df400mov  = pd.read_csv(mri_path400 + movie_name400)
    movies = [m for m in mri_df400mov.filename.unique() if "414" in m] 
    subject_idx = [int(m[12:14]) for m in movies]
    order_idx[movie_name400.split('_')[0]] = subject_idx


  0%|          | 0/15 [00:00<?, ?it/s]

In [19]:
order_idx['Sintel']

{'TearsOfSteel': {...},
 'Superhero': {...},
 'BetweenViewing': {...},
 'BigBuckBunny': {...},
 'Sintel': {...},
 'Payload': {...},
 'AfterTheRain': {...},
 'TheSecretNumber': {...},
 'FirstBite': {...},
 'LessonLearned': {...},
 'ToClaireFromSonny': {...},
 'Chatter': {...},
 'Spaceman': {...},
 'YouAgain': {...}}

In [ ]:
load('./resources/static_grad/individual_level/AfterTheRain_grad414.pkl')

In [11]:
fselect = load('./resources/threshes/tresh40-0.pkl')


(30, 406, 414)

In [12]:
load('./resources/threshes/tresh40-0.pkl')['TearsOfSteel'].shape

(30, 195, 414)

In [6]:
# Params
embedding  = "dm" # diffusion map
aff_kernel = "pearson" #affinity matrix kernel
align_meth = "procrustes"
n_iter     = 10 # procrustes align number of iteration
nb_comp    = 3
rs         = 99

1. Frame selection compute

In [8]:
# individual variance variables
fselect_subjects_gradients = {}

# bootstrap variables
sample_size = 25
resample_nb = 100
fselect_bootstrap_gradients = {}

np.random.seed(19)
for tpath in os.listdir("./resources/threshes/"):
    selected_series = load('./resources/threshes/{}'.format(tpath))

    # concatenate on different frames across movies
    # basically doing reordering of columns
    subject_tmpseries = [[] for _ in range(30)]
    tmp = list(selected_series.values())
    for i in range(len(tmp)):
        if len(tmp[i]) == 0: continue
        for nsub in range(30):
            subject_tmpseries[nsub].append(tmp[i][nsub])

    subject_series = []
    for i in range(30):
        subject_series.append(np.concatenate(subject_tmpseries[i]))

    # 1. Compute individual gradients according to selected frames
    subjects_gradients = []
    for nsub in range(30):
        tmp_series = np.asarray(subject_series)[nsub]
        fc_selects = FC(tmp_series)
        g_selects = GradientMaps(n_components=nb_comp, approach=embedding, 
                            kernel=aff_kernel, random_state=rs)
        g_selects.fit(fc_selects)
        subjects_gradients.append(g_selects.gradients_)

    # alignement across all gradients
    select_subjects_gradients,_,s = procrustes_align(np.asarray(subjects_gradients))
    select_subjects_gradients = np.asarray(select_subjects_gradients)
    fselect_subjects_gradients[tpath.split('.')[0]] = select_subjects_gradients


    # 2. Compute bootrstrap group gradients according to selected frames
    bootstrap_gradients = []
    for rnb in range(resample_nb):
        index_choice = np.random.choice(np.arange(30), sample_size, replace=False)
        tmp_series = np.concatenate(np.array(subject_series)[index_choice])
        fc_selects = FC(tmp_series)
        g_selects = GradientMaps(n_components=nb_comp, approach=embedding, 
                            kernel=aff_kernel, random_state=rs)
        g_selects.fit(fc_selects)
        bootstrap_gradients.append(g_selects.gradients_)
    # alignement across all gradients
    select_bootstrap_gradients,_,s = procrustes_align(np.asarray(bootstrap_gradients))
    select_bootstrap_gradients = np.asarray(select_bootstrap_gradients)
    fselect_bootstrap_gradients[tpath.split('.')[0]] = select_bootstrap_gradients